In [1]:
# Advent of Code 2023
# Day 20 Problem 1
import re
from collections import Counter
import functools

with open("aoc_20_input.txt") as f:
   A =  f.read().strip().split("\n")

# c+p test case here
TEST = """broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a""".strip().split("\n")

TESTB = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output""".strip().split("\n")

In [2]:
"""
% Flip flop modules are either ON or OFF. 
    - initially off
    - HIGH pulse == ignored and nothing happens
    - LOW pules == flip between ON and OFF
        - if OFF, turn ON and send HIGH pulse
        - if ON, turn OFF and send LOW pulse

& conjunction modules remember the type of the most recent pulse received from each of their connected input modules
    - initially remember LOW pulses from each input
    - pulse received --> update memory for that input
        - then, if it remembers HIGH for all inputs, send LOW pulse
        - otherwise send HIGH pulse

broadcast module
    - when it receives a pulse, sends same pulse to all of its destination modules

button modules
    - sends a LOW pulse to broadcaster

"""

'\n% Flip flop modules are either ON or OFF. \n    - initially off\n    - HIGH pulse == ignored and nothing happens\n    - LOW pules == flip between ON and OFF\n        - if OFF, turn ON and send HIGH pulse\n        - if ON, turn OFF and send LOW pulse\n\n& conjunction modules remember the type of the most recent pulse received from each of their connected input modules\n    - initially remember LOW pulses from each input\n    - pulse received --> update memory for that input\n        - then, if it remembers HIGH for all inputs, send LOW pulse\n        - otherwise send HIGH pulse\n\nbroadcast module\n    - when it receives a pulse, sends same pulse to all of its destination modules\n\nbutton modules\n    - sends a LOW pulse to broadcaster\n\n'

In [3]:
from collections import deque, defaultdict
import re

class PulseSystem:
    # init method reading from input
    # queue of pulses sent

    def __init__(self, input) -> None:
        self.buttonPresses = 0
        self.pulseQueue = deque()
        self.modules = defaultdict()
        for line in input:
            #print(f"\t{line}")
            c = line[0]
            if c == "%":
                F = FlipFLop(line)
                self.modules[F.name] = F
            elif c == "&":
                C = Conjunction(line)
                self.modules[C.name] = C
            else:
                B = Broadcaster(line)
                self.modules["broadcaster"] = B
            #self.printSystem()
        # also need to keep track of conjunction inputs
        conjs = {line.split(" -> ")[0][1:]: [] for line in input if line[0] == "&"}
        for line in input:
            source, dests = line.split(" -> ")
            for c in conjs.keys():
                if c in dests:
                    conjs[c].append(source[1:].strip())
        for c in conjs:
            self.modules[c].setInputs(conjs[c])

    def pressButton(self):
        self.buttonPresses += 1
        self.pulseQueue.append(["broadcaster", False, "button"])
        while self.pulseQueue:
            #self.printPulse(self.pulseQueue[0])
            #print(self.pulseQueue)
            dest, pulse, source = self.pulseQueue.popleft()
            if dest in self.modules.keys():
                self.pulseQueue.extend(self.modules[dest].handlePulse(pulse, source))
    
    def countPulses(self):
        low = self.buttonPresses
        high = 0
        for module in self.modules.values():
            low += module.sentLowPulses
            high += module.sentHighPulses
        return low, high, low*high
    
    def printSystem(self):
        for module in self.modules.values():
            print(f"{module.name} -> {module.destModules}")
        print()

    def printPulse(self, p):
        print(f"{p[2]} -{'high' if p[1] else 'low'}-> {p[0]}")


class Broadcaster:    
    def __init__(self, line) -> None:
        self.name = "broadcaster"
        self.sentLowPulses = 0
        self.sentHighPulses = 0
        self.destModules = []
        self.destLen = 0
        result = line.split(" -> ")[1]
        if "," in result:
            for mod in result.split(","):
                self.destModules.append(mod.strip())
        else:
            self.destModules.append(result.strip())
        self.destLen = len(self.destModules)

    def handlePulse(self, pulse, source=None):
        # p = 0 -> low
        # p = 1 -> high
        res = []
        if pulse == 0:
            for dest in self.destModules:
                res.append([dest, False, self.name])
            self.sentLowPulses += self.destLen
        else:
            for dest in self.destModules:
                res.append([dest, True, self.name])
            self.sentHighPulses += self.destLen
        return res

    def prettyprint(self):
        print(f"{self.name} -> {self.destModules}")

class FlipFLop:    
    def __init__(self, line) -> None:
        self.on = False
        self.sentLowPulses = 0
        self.sentHighPulses = 0
        self.destModules = []
        self.destLen = 0

        # %a -> inv, con
        first, second = line.split(" -> ")
        self.name = first[1:].strip()
        if "," in second:
            for mod in second.split(","):
                self.destModules.append(mod.strip())
        else:
            self.destModules.append(second.strip())
        self.destLen = len(self.destModules)

    def handlePulse(self, pulse, source=None):
        # p = 0 -> low
        # p = 1 -> high
        if pulse == 1:
            return []
    
        res = []
        self.on = not self.on
        for dest in self.destModules:
            # destination, low/high, source
            res.append([dest, self.on, self.name])

        if self.on:
            self.sentHighPulses += self.destLen
        else:
            self.sentLowPulses += self.destLen

        return res



class Conjunction:
    def __init__(self, line) -> None:
        self.on = False
        self.name = ""
        self.sentLowPulses = 0
        self.sentHighPulses = 0
        self.destModules = []
        self.destLen = 0
        self.lastPulses = defaultdict()
        
        first, second = line.split(" -> ")
        self.name = first[1:].strip()
        if "," in second:
            for mod in second.split(","):
                self.destModules.append(mod.strip())
        else:
            self.destModules.append(second.strip())
        self.destLen = len(self.destModules)

    def setInputs(self, inp):
        for i in inp:
            self.lastPulses[i] = False

    def handlePulse(self, pulse, source):
        # p = 0 -> low
        # p = 1 -> high
        
        # update memory
        # then if all high, send LOW
        # otherwise send HIGH
        self.lastPulses[source] = pulse
        res = []

        if False in self.lastPulses.values():
            for dest in self.destModules:
                res.append([dest, True, self.name])
            self.sentHighPulses += self.destLen
        else:
            for dest in self.destModules:
                res.append([dest, False, self.name])
            self.sentLowPulses += self.destLen

        return res


def p1(input):
    system = PulseSystem(input)
    #system.printSystem()
    for _ in range(1000):
        system.pressButton()
    return system.countPulses()[2]

In [4]:
# expected value: 32000000
p1(TEST)

32000000

In [5]:
# expected value: 11687500
p1(TESTB)

11687500

In [6]:
p1(A)

839775244

In [7]:
from functools import reduce
class PulseSystemP2:
    # init method reading from input
    # queue of pulses sent

    def __init__(self, input) -> None:
        self.buttonPresses = 0
        self.pulseQueue = deque()
        self.zhInputs = {
            "vd": 0,
            "ns": 0,
            "bh": 0, 
            "dl": 0
        }
        self.modules = defaultdict()
        for line in input:
            c = line[0]
            if c == "%":
                F = FlipFLop(line)
                self.modules[F.name] = F
            elif c == "&":
                C = Conjunction(line)
                self.modules[C.name] = C
            else:
                B = Broadcaster(line)
                self.modules["broadcaster"] = B
            #self.printSystem()
        # also need to keep track of conjunction inputs
        conjs = {line.split(" -> ")[0][1:]: [] for line in input if line[0] == "&"}
        for line in input:
            source, dests = line.split(" -> ")
            for c in conjs.keys():
                if c in dests:
                    conjs[c].append(source[1:].strip())
        for c in conjs:
            self.modules[c].setInputs(conjs[c])

    def pressButton(self):
        self.buttonPresses += 1
        self.pulseQueue.append(["broadcaster", False, "button"])
        while self.pulseQueue:

            dest, pulse, source = self.pulseQueue.popleft()

            if dest == "zh" and pulse == 1:
                if self.zhInputs[source] == 0:
                    self.zhInputs[source] = self.buttonPresses

                if 0 not in self.zhInputs.values():
                    return reduce(lambda x, y: x*y, self.zhInputs.values())
                
            if dest in self.modules.keys():
                self.pulseQueue.extend(self.modules[dest].handlePulse(pulse, source))
        return 0
    
    def printSystem(self):
        for module in self.modules.values():
            print(f"{module.name} -> {module.destModules}")
        print()

    def printPulse(self, p):
        print(f"{p[2]} -{'high' if p[1] else 'low'}-> {p[0]} [button press {self.buttonPresses}]")


def p2(input):
    system = PulseSystemP2(input)
    #system.printSystem()
    res = 0
    while not res:
        res = system.pressButton()
    return res

In [8]:
p2(A)

207787533680413